In [2]:
import boto3

sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()["Account"]
my_session = boto3.session.Session()
region = my_session.region_name
print(account_id)

224863548990


In [3]:
prefix = "lidar-data"
bucket = "sagemaker-"+str(region)+"-"+str(account_id)

In [4]:
from sagemaker import get_execution_role
try: 
    role = get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_role')['Role']['Arn']

Couldn't call 'get_role' to get Role ARN from role name sagemaker_local to get Role path.


In [5]:
client = boto3.client("sts")
account = client.get_caller_identity()["Account"]

region = my_session.region_name

algorithm_name = "lidar"

ecr_image = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, algorithm_name)

In [6]:
data_location = "s3://{}/{}".format(bucket,prefix) 

In [7]:
from sagemaker.estimator import Estimator

hyperparameters = {"epochs": 1}

instance_type = "ml.m4.xlarge"

estimator = Estimator(
    role=role,
    train_instance_count=1,
    train_instance_type=instance_type,
    image_uri=ecr_image,
    hyperparameters=hyperparameters,
)

estimator.fit(data_location)

predictor = estimator.deploy(1, instance_type)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: No S3 objects found under S3 URL "s3://sagemaker-us-east-1224863548990/lidar-data" given in input data source. Please ensure that the bucket exists in the selected region (us-east-1), that objects exist under that S3 prefix, and that the role "arn:aws:iam::224863548990:role/sagemaker_role" has "s3:ListBucket" permissions on bucket "sagemaker-us-east-1224863548990". Error message from S3: The specified bucket does not exist

In [ ]:
import torchvision, torch
import numpy as np

from utils.utils_cifar import get_test_data_loader, imshow, classes
testloader = get_test_data_loader("/tmp/lidar")

from sagemaker.predictor import json_serializer, json_deserializer

In [ ]:
# get some test images
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print("GroundTruth: ", " ".join("%4s" % classes[labels[j]] for j in range(4)))

# predictor.accept = "application/json"
# predictor.content_type = "application/json"

predictor.serializer = json_serializer
predictor.deserializer = json_deserializer

outputs = predictor.predict(images.numpy())

_, predicted = torch.max(torch.from_numpy(np.array(outputs)), 1)

print("Predicted: ", " ".join("%4s" % classes[predicted[j]] for j in range(4)))

In [ ]:
predictor.delete_endpoint()